<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">

# Desafío 1

# ¡Listo para corregir!

## Alumno
Denardi, Fabricio

## Cohorte
15-2024

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [454]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import random

## Carga de datos

In [455]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [456]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer(stop_words='english')

In [457]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [458]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [459]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101322)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101322


In [460]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25717

In [461]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [462]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [463]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [464]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [465]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [466]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.5377831 , 0.49482167, ..., 0.        , 0.        ,
       0.        ])

In [467]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([4811, 6635, 3596, ..., 5915, 5925, 1369])

In [468]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [469]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [470]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [471]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [472]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [473]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.6467991505900852

# Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


# Resolución del desafío

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

In [474]:
def get_sample_elements(origin:list, qty:int = 5, seed:int=4567) -> list:
    '''Obtiene n elementos al azar en la lista origen dada'''
    random.seed(seed)
    return random.sample(origin,qty)

In [475]:
train_data = newsgroups_train.data

In [476]:
sample_documents_idx_list = get_sample_elements(range(len(train_data)),5)

In [477]:
sample_documents_idx_list

[7030, 8869, 8483, 4807, 121]

In [478]:
def analize_similarity (idx_list: list,  X_train:np.ndarray, y_train:np.ndarray, data:dict, target_names:dict, top_similarity:int = 5, print_res:bool=True, entity_to_analize:str= "documento"):
    '''Analiza los documentos o términos cuyos índices son pasados como parámetro y su similaridad con el resto de los documentos o términos según se esté analizando'''

    for document_ix in idx_list:
        print('==============================================================================================')
        print('==============================================================================================')
        print(f'El {entity_to_analize} analizado es el índice {document_ix}.')

        if target_names is not None:
          print(f'La categoría del mismo es: "{target_names[y_train[document_ix]]}". ')
      
        if print_res:
          document = data[document_ix]
          print (f'Longitud: {len(document)}')
          print(f'El {entity_to_analize} es el siguiente:')
          print(document)


        # Medir la similaridad coseno con todas la entidades de train
        cossim = cosine_similarity(X_train[document_ix], X_train)[0]

        # Ordenar los valores de similitud de mayor a menor
        np.sort(cossim)[::-1]

        # Las n entidades más similares:
        most_sim = np.argsort(cossim)[::-1][1:top_similarity + 1] #El primero no se tiene en cuenta ya que se trata de la entidad analizada (tiene 1 de similiradidad ya que es idéntico -es el mismo-)

        # Analizar dichos elementos
        for sim_ix,pattern_ix in enumerate(most_sim):
            print('-------------------------------------------------------------------------------------------')
            print(f'»» {entity_to_analize} con similaridad - Top {sim_ix + 1}')
            print(f'Tiene el índice {pattern_ix}.')

            if target_names is not None:
              print(f'La categoría del mismo es: "{target_names[y_train[pattern_ix]]}".')
                    
            if print_res:
              sim_document = data[pattern_ix]
              print (f'Longitud: {len(sim_document)}')
              print(f'El {entity_to_analize} es el siguiente:')
              print(sim_document)

In [479]:
analize_similarity(sample_documents_idx_list,X_train,y_train,newsgroups_train.data,newsgroups_train.target_names,5,False,entity_to_analize="documento")

El documento analizado es el índice 7030.
La categoría del mismo es: "sci.electronics". 
-------------------------------------------------------------------------------------------
»» documento con similaridad - Top 1
Tiene el índice 3668.
La categoría del mismo es: "sci.electronics".
-------------------------------------------------------------------------------------------
»» documento con similaridad - Top 2
Tiene el índice 652.
La categoría del mismo es: "sci.electronics".
-------------------------------------------------------------------------------------------
»» documento con similaridad - Top 3
Tiene el índice 2236.
La categoría del mismo es: "sci.electronics".
-------------------------------------------------------------------------------------------
»» documento con similaridad - Top 4
Tiene el índice 3014.
La categoría del mismo es: "sci.electronics".
-------------------------------------------------------------------------------------------
»» documento con similaridad - T

Dado que el costo computacional de estos algortimos para el input dado no es demasiado, en la celda anterior, mostré solo los resultados simplificados, es decir sin analizar el documento, ahora lo ejecutaré de nuevo pero visualizando los documentos, para analizar los resultados con más detalle y en el contextos de los mismos. 

In [480]:
analize_similarity(sample_documents_idx_list,X_train,y_train,newsgroups_train.data,newsgroups_train.target_names,5,True,entity_to_analize="documento")

El documento analizado es el índice 7030.
La categoría del mismo es: "sci.electronics". 
Longitud: 647
El documento es el siguiente:
Yes, I know it sounds crazy. Call it an urge. Call it what you want.
Just don't ask why :-)

Anyway, I'd like to build a UV flashlight, cheaply. "flashlight" means
it should be reasonably portable, but could have a power pack if
necessary.

My main question is the bulb: where can I get UV bulbs? Do they
need a lot of power? etc., etc.

I'm not too concerned with whether it's long-wave or short-wave
(but hey, if anyone has a cheap source of bulbs, I'll take both).

One other thing: a friend of mine mentioned something about near-UV
light being cheaper to get at than actual UV light. Does anyone
know what he was referring to?

Thanks much.


-------------------------------------------------------------------------------------------
»» documento con similaridad - Top 1
Tiene el índice 3668.
La categoría del mismo es: "sci.electronics".
Longitud: 1159
El docu

#### Conclusiones del ejercicio
1. Algunos documentos son más propensos a que, aplicando el algoritmo de similaridad por coseno, tengan otros documentos similares, de la misma categoría.

2. A pesar de que algunos matches no pertenecen a la misma categoría, si nos ponemos a leer, podemos encontrar tópicos o temas parecidos. Lo cual le encuentro sentido. Por ejemplo, en el caso analizado de comp.os.ms-windows.misc, habla de un magazine de tecnlogía, y discute sobre temas de hardware, gráficos, y la similitud del coseno encuentra justamente documentos que se encuadran en esas categorías.

3. Si bien la bibliografía de scikit-learn advierte del uso del parámetro stop_word, dado que los textos están en inglés y el único valor válido justamente es dicho idioma, probé parametrizarlo y considero que obtuve una pequeña mejora. 

4. Aplicar n-gramas también consideor que fue satisfactorio aunque no noté demasiada mejoría, además luego en el ejercicio de la matriz trampuesta (para ver la relación de palabras, no tenía el efecto deseado)

5. Intenté "jugar" con los valores de min_df y max_df, estableciendo diferentes rangos de threshold, pero en este caso no tuve éxito, ya que rangos muy restrictivos me eliminaban por completo la salida, dando el siguiente error: "After pruning, no terms remain. Try a lower min_df or a higher max_df.". En otros casos me eliminaba textos de categorías o palabras del corpus, como por ej "car" que justo era la que estaba de ejemplo.

4. Este algoritmo no captura sinónimos, es decir frases o documentos que son semánticamente similares pero que no comparten términos, la similaridad del coseno nos va a dar cero. Es por eso que necesitaremos modelos más complejos, como embeddings, que desarrollaremos en el próximo desafío.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

In [481]:
def analize_naive_bayes(model:object,average='macro'):
    '''Analiza la performance del modelo '''
    model.fit(X_train, y_train)

    X_test = tfidfvect.transform(newsgroups_test.data)
    y_test = newsgroups_test.target
    y_pred =  model.predict(X_test)

    accuracy = f1_score(y_test, y_pred, average='macro')
    
    print(f'El F1 score con average {average} para el modelo {type(model).__name__} es: {accuracy:.4f}')

In [482]:
naive_classifier = MultinomialNB()
print("Naive Bayes - Parámetros por defecto")
analize_naive_bayes(naive_classifier)

Naive Bayes - Parámetros por defecto
El F1 score con average macro para el modelo MultinomialNB es: 0.6468


In [483]:
naive_classifier = MultinomialNB(alpha=0.01, force_alpha=True, fit_prior=False, class_prior=None)
print("Naive Bayes - Fine tuning")
analize_naive_bayes(naive_classifier)

Naive Bayes - Fine tuning
El F1 score con average macro para el modelo MultinomialNB es: 0.6877


In [484]:
naive_classifier = ComplementNB()
print("Complement Naive Bayes - Parámetros por defecto")
analize_naive_bayes(naive_classifier)

Complement Naive Bayes - Parámetros por defecto
El F1 score con average macro para el modelo ComplementNB es: 0.6936


In [485]:
naive_classifier = ComplementNB(alpha=0.1, force_alpha=True, fit_prior=False, class_prior=None, norm=False)
print("Complement Naive Bayes - Fine tuning")
analize_naive_bayes(naive_classifier)

Complement Naive Bayes - Fine tuning
El F1 score con average macro para el modelo ComplementNB es: 0.6919


#### Conclusiones del ejercicio
1. Realizar un fine tuning de los parámetros de los modelos resultó una buena elección. Proporcionalmente, tuvo mayor efecto en el Naive Bayes que en el Complement Naive Bayes.

2. El mejor modelo obtenido fue el ComplementNB ajustando los valores de los parámetros.

3. Como comentario personal, creo que el mejor F1 score obtenido, no resulta suficiente. Por ejemplo, yo trabajo para un Broker de seguros, y tenemos diferentes tipos de documentos que usualmente necesitamos clasificar. Este score no resulta prometedor, ya que puede genera problemas regulatorios o inclusos legales, ya que el rubro de seguros es muy incumbente y normado, que se rige por muchímas regulaciones.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

In [486]:
X_train.shape

(11314, 101322)

In [487]:
X_train.T.shape

(101322, 11314)

In [488]:
sample_terms_list = ['car','bear','light','green','star']

In [489]:
sample_terms_idx_list = []

for term in sample_terms_list:
    sample_terms_idx_list.append(tfidfvect.vocabulary_[term])

In [490]:
analize_similarity(sample_terms_idx_list,X_train.T,None,idx2word,None,5,True,entity_to_analize="término")

El término analizado es el índice 25717.
Longitud: 3
El término es el siguiente:
car
-------------------------------------------------------------------------------------------
»» término con similaridad - Top 1
Tiene el índice 25863.
Longitud: 4
El término es el siguiente:
cars
-------------------------------------------------------------------------------------------
»» término con similaridad - Top 2
Tiene el índice 30471.
Longitud: 9
El término es el siguiente:
criterium
-------------------------------------------------------------------------------------------
»» término con similaridad - Top 3
Tiene el índice 32086.
Longitud: 6
El término es el siguiente:
dealer
-------------------------------------------------------------------------------------------
»» término con similaridad - Top 4
Tiene el índice 27504.
Longitud: 5
El término es el siguiente:
civic
-------------------------------------------------------------------------------------------
»» término con similaridad - Top 5


#### Conclusiones del ejercicio
1. A grande rasgos, podemos decir que este método es **medianamente satisfactorio** para encontrar términos similares. 

2. Esta similaridad depende exclusivamente del contexto y la semántica de los documentos del corpus que poseemos. Por ejemplo, para la palabra *star*, el término más similar encontrado es *trek* que forman *star trek*, una reconocida serie de televisión. O para *car*, obtenemos *dealer* como uno de sus términos similares, que se refiere a los concesionarios de autos.

3. En otros caso, como *car* también obtenemos su plural como término, lo cuál también tiene sentido.

4. Lo que a mí me resultó contraintuitivo, es que el algoritmo, encontró pocos sinónimos o palabras dentro de una misma "especie", por ejemplo para "green", en el top-5 recién apareció otro color, *yellow*. Tratando de entender el porque, llegué a la conclusión que está correcto, ya que este algoritmo no comprende de sinónimos, antónimos, grupo de palabras, sino como se relacionan los términos en los documento del corpus y en los contextos de estos, como por ejemplo *green urbina*, que puede ser un ex jugador de beseball, cuya camiseta del equipo que lo hizo estrella era verde o el nombre de una planta.

5. El punto anterior considero que se debe a que estos modelos no interpretan el contexto, es decir, en dónde esta la palabra y sus término vecinos.